# 综合练习一

In [356]:
import numpy as np
import pandas as pd
df1=pd.read_csv('D:\datawhale\joyful-pandas\data\exercise\company.csv')
df2=pd.read_csv('D:\datawhale\joyful-pandas\data\exercise\company_data.csv')

In [357]:
df1['证券代码']=df1['证券代码'].apply(lambda x:x.strip('#').lstrip('0'))
df1.head()

,证券代码,日期
0,7,2014
1,403,2015
2,408,2016
3,408,2017
4,426,2015


In [358]:
df2['年份']=df2['日期'].str.split('/').apply(lambda x:int(x[0]))
df2.set_index(['证券代码','年份']).head(3)

日期  收入类型           收入额
证券代码 年份                                  
1    2008  2008/12/31     1  1.084218e+10
     2008  2008/12/31     2  1.259789e+10
     2008  2008/12/31     3  1.451312e+10

In [359]:
import warnings
warnings.filterwarnings('ignore')

res=np.full((len(df1['证券代码']),1),np.nan)
for i in range(len(df1['证券代码'])):
    h=df1['证券代码'].loc[i]
    y=df1['日期'].loc[i]
    ind=df2.query("证券代码==@h & 年份==@y").index
    temp=df2.loc[ind.tolist()]
    temp2=temp[temp['收入额']!=0]
    all_=temp2.loc[:,'收入额'].sum(axis=0)
    temp2.loc[:,'收入比重']=temp2.loc[:,'收入额']/all_
    temp2['收入熵']=-temp2['收入比重']*np.log10(temp2['收入比重'])
    res[i]=temp2['收入熵'].sum()
    
print(res)

[[1.33348471]
 [1.21193566]
 [1.22407682]
 ...
 [1.30836766]
 [1.31207184]
 [0.        ]]


In [360]:
df1['收入熵']=res
df1.head()

,证券代码,日期,收入熵
0,7,2014,1.333485
1,403,2015,1.211936
2,408,2016,1.224077
3,408,2017,0.000000
4,426,2015,1.339480


# 综合练习二

In [361]:
df1=pd.read_excel('D:\datawhale\joyful-pandas\data\exercise\组队信息汇总表（Pandas）.xlsx')

In [362]:
columns1=['所在群','队伍名称','编号_1','1']
columns2=['编号_'+str(i)+'-'+str(i) for i in range(2,12)]
columns2=sum([columns2[i].split('-') for i in range(10)],[])
columns1.extend(columns2)
df1.columns=columns1
df1.head(1)

,所在群,队伍名称,编号_1,1,编号_2,2,编号_3,3,编号_4,4,...,编号_7,7,编号_8,8,编号_9,9,编号_10,10,编号_11,11
0,Pandas数据分析,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [363]:
res=pd.wide_to_long(df1,stubnames='编号',i=['队伍名称','所在群'],j='是否队长',sep='_',suffix='.+').reset_index()
res2=res[~np.isnan(res['编号'])]
res2.index=[i for i in range(res2.shape[0])]
res2.head(3)

,队伍名称,所在群,是否队长,8,4,10,2,7,9,5,6,3,11,1,编号
0,你说的都对队,Pandas数据分析,1,NaN,信仰,NaN,蔡,NaN,NaN,biubiu🙈🙈,NaN,安慕希,NaN,山枫叶纷飞,5.0
1,你说的都对队,Pandas数据分析,2,NaN,信仰,NaN,蔡,NaN,NaN,biubiu🙈🙈,NaN,安慕希,NaN,山枫叶纷飞,6.0
2,你说的都对队,Pandas数据分析,3,NaN,信仰,NaN,蔡,NaN,NaN,biubiu🙈🙈,NaN,安慕希,NaN,山枫叶纷飞,7.0


In [364]:
temp=res2.iloc[:,2:-1]
for i in range(res2.shape[0]):
    tempi=temp.iloc[i]
    tn=temp[str(temp['是否队长'].iloc[i])][i]
    temp.loc[i,:]=tempi.mask(tempi!=tn,'NaN')
    
temp.head(3)

,是否队长,8,4,10,2,7,9,5,6,3,11,1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,山枫叶纷飞
1,NaN,NaN,NaN,NaN,蔡,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,安慕希,NaN,NaN


In [365]:
res2.iloc[:,3:-1]=temp.iloc[:,1::]
res2.head(3)

,队伍名称,所在群,是否队长,8,4,10,2,7,9,5,6,3,11,1,编号
0,你说的都对队,Pandas数据分析,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,山枫叶纷飞,5.0
1,你说的都对队,Pandas数据分析,2,NaN,NaN,NaN,蔡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
2,你说的都对队,Pandas数据分析,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,安慕希,NaN,NaN,7.0


In [373]:
columns1=['队伍名称','所在群','是否队长']
columns2=['群昵称_'+str(i) for i in range(1,12)]
columns1.extend(columns2)
columns1.extend(['编号'])
res2.columns=columns1
res2.head(3)

,队伍名称,所在群,是否队长,群昵称_1,群昵称_2,群昵称_3,群昵称_4,群昵称_5,群昵称_6,群昵称_7,群昵称_8,群昵称_9,群昵称_10,群昵称_11,编号
0,你说的都对队,Pandas数据分析,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,山枫叶纷飞,5.0
1,你说的都对队,Pandas数据分析,2,NaN,NaN,NaN,蔡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
2,你说的都对队,Pandas数据分析,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,安慕希,NaN,NaN,7.0


In [375]:
res3=pd.wide_to_long(res2,stubnames='群昵称',i=['队伍名称','所在群','是否队长'],j='昵称',sep='_',suffix='.+').reset_index()
res4=res3[res3['群昵称']!='NaN'].drop(columns=['所在群','昵称'])
res4['是否队长']=res4['是否队长'].mask(res4['是否队长']!=1,0)
res4.index=[i for i in range(res4.shape[0])]
res4=res4.reindex(columns=['是否队长','队伍名称','群昵称','编号'])
res4.head(3)

,是否队长,队伍名称,群昵称,编号
0,1,你说的都对队,山枫叶纷飞,5.0
1,0,你说的都对队,蔡,6.0
2,0,你说的都对队,安慕希,7.0


# 综合练习三

- 有多少县满足总投票数超过县人口数的一半

In [435]:
df1=pd.read_csv('D:\datawhale\joyful-pandas\data\exercise\county_population.csv')
df2=pd.read_csv('D:\datawhale\joyful-pandas\data\exercise\president_county_candidate.csv')

In [437]:
df1.head(3)

,US County,Population
0,".Autauga County, Alabama",55869
1,".Baldwin County, Alabama",223234
2,".Barbour County, Alabama",24686


In [439]:
df1.shape

(3142, 2)

In [440]:
df1['US County']=df1['US County'].apply(lambda x:x.strip('.'))#取出前面的小点点
df1['county']=df1['US County'].map(lambda x:x.split(',')[0])
df1['state']=df1['US County'].map(lambda x:x.split(',')[1])
df1.head(3)

,US County,Population,county,state
0,"Autauga County, Alabama",55869,Autauga County,Alabama
1,"Baldwin County, Alabama",223234,Baldwin County,Alabama
2,"Barbour County, Alabama",24686,Barbour County,Alabama


In [444]:
tv=df2.groupby('county')['total_votes'].sum().reset_index()
res=df1.merge(tv,on=['county'],how='left')
res[res['total_votes']>(res['Population']/2)].shape[0]

2292

有2292个县满足总投票数超过县人口数的一半

- 把州（state）作为行索引，把投票候选人作为列名，列名的顺序按照候选人在全美的总票数由高到低排序，行列对应的元素为该候选人在该州获得的总票数

In [462]:
res=df2.pivot_table(index='state', columns='candidate', values='total_votes',aggfunc='sum')
columns=res.sum(0).sort_values(ascending=False).index
res=res.reindex(columns=columns.tolist())
res.head(3)

candidate,Joe Biden,Donald Trump,Jo Jorgensen,Howie Hawkins,Write-ins,Rocky De La Fuente,Gloria La Riva,Kanye West,Don Blankenship,Brock Pierce,...,Tom Hoefling,Ricki Sue King,Princess Jacob-Fambro,Blake Huber,Richard Duncan,Joseph Kishore,Jordan Scott,Gary Swing,Keith McCormic,Zachary Scalf
state,,,,,,,,,,,,,,,,,,,,,
Alabama,849648.0,1441168.0,25176.0,NaN,7312.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alaska,153405.0,189892.0,8896.0,NaN,34210.0,318.0,NaN,NaN,1127.0,825.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,1672143.0,1661686.0,51465.0,NaN,2032.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 每一个州下设若干县，定义拜登在该县的得票率减去川普在该县的得票率为该县的BT指标，若某个州所有县BT指标的中位数大于0，则称该州为Biden State，请找出所有的Biden State

In [465]:
df2.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [524]:
res2=df2.pivot_table(index='county', columns='candidate', values='total_votes',aggfunc='sum')
res2['BT']=res2['Joe Biden']-res2['Donald Trump']
res2=res2.reset_index()
res_temp=res2.iloc[:,[0,-1]]#取出county与BT数丢回去
res3=df2.merge(res_temp,on=['county'],how='left')
res4=res3.drop_duplicates(['state','county'],keep='first')#删除冗余的行
final=res4.groupby('state')['BT'].median()
final[np.where(final>0)[0].tolist()].index

Index(['California', 'Connecticut', 'Delaware', 'District of Columbia',
       'Hawaii', 'Massachusetts', 'New Hampshire', 'New Jersey',
       'Rhode Island', 'Vermont'],
      dtype='object', name='state')